In [ ]:
# ========================================
# 📚 1. Introduction : Word2Vec - Skip-Gram (avec One-Hot)
# ========================================

"""
👋 Bienvenue !

🎯 Objectif :
- Créer un Word2Vec de type Skip-Gram avec softmax
- Utiliser one-hot encoding pour voir les mécanismes internes
- Comprendre l’apprentissage de représentations vectorielles

🎒 À savoir :
- On utilisera une phrase simple
- Encodage one-hot à la main
- Implémentation explicite des matrices poids
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
# ========================================
# ✏️ 2. Corpus de base
# ========================================

"""
➡️ Entrée : une phrase
➡️ Sortie : vocabulaire + indexation + paires (centre, contexte)
"""

text = "the cat sat on the mat"
words = text.split()
vocab = sorted(set(words))
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}

print("📚 Vocabulaire :", word_to_ix)
print("🧮 Taille du vocabulaire :", vocab_size)

In [ ]:
# ========================================
# 🔁 3. Générer les paires Skip-Gram
# ========================================

"""
🎯 Objectif : Créer les paires (input word, context word)

🔧 À faire :
- Complète la boucle pour que chaque mot génère ses voisins
- Modifier window_size pour observer la différence
"""

def generate_skipgram_pairs(words, window_size=1):
    pairs = []
    for i, center_word in enumerate(words):
        for j in range(i - window_size, i + window_size + 1):
            if j != i and 0 <= j < len(words):
                context_word = words[j]
                pairs.append((center_word, context_word))
    return pairs

pairs = generate_skipgram_pairs(words, window_size=1)

print("📦 Paires (centre, contexte) :")
for p in pairs:
    print(p)

In [ ]:
# ========================================
# 🧱 4. Encodage One-Hot
# ========================================

"""
➡️ On encode chaque mot en vecteur one-hot

🔧 À faire :
- Complète la fonction one_hot_encode
- Vérifie que l'encodage est correct
"""

def one_hot_encode(word, word_to_ix):
    vector = np.zeros(len(word_to_ix))
    index = word_to_ix[word]
    vector[index] = 1
    return vector

# Exemple :
print(f"🧪 'cat' → {one_hot_encode('cat', word_to_ix)}")

In [ ]:
# ========================================
# 🧠 5. Initialiser les poids du modèle
# ========================================

"""
➡️ On va simuler le modèle Word2Vec :
- 1 couche cachée : W1 (vocab_size x emb_dim)
- 1 couche de sortie : W2 (emb_dim x vocab_size)

🔧 À faire :
- Choisir la dimension d'embedding
- Initialiser W1 et W2 aléatoirement
"""

embedding_dim = 5

# 🧱 Matrices de poids
W1 = np.random.randn(vocab_size, embedding_dim)
W2 = np.random.randn(embedding_dim, vocab_size)

print("✅ W1 shape:", W1.shape)
print("✅ W2 shape:", W2.shape)

In [ ]:
# ========================================
# 🔥 6. Fonction Softmax
# ========================================

"""
➡️ Transforme un vecteur de scores en distribution de probabilités
"""

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stabilise les valeurs
    return e_x / np.sum(e_x)

In [ ]:
# ========================================
# 🧮 7. Forward pass : prédire contexte
# ========================================

"""
➡️ Implémenter la passe avant :
- Encode le mot central en one-hot
- Multiplie par W1 → vecteur caché
- Multiplie par W2 → scores pour tous les mots
- Applique softmax

🔧 À faire :
- Complète la fonction forward
"""

def forward(word, W1, W2):
    x = one_hot_encode(word, word_to_ix)       # One-hot
    h = np.dot(x, W1)                          # Projection en espace dense
    u = np.dot(h, W2)                          # Scores bruts
    y_pred = softmax(u)                        # Probabilités
    return y_pred, h

# Test
probs, hidden = forward("cat", W1, W2)
print(f"📊 Proba prédite pour 'cat' : {np.round(probs, 3)}")
print("🧠 Vecteur caché :", hidden)

In [ ]:
# ========================================
# 🎓 8. Fonction de perte + backpropagation
# ========================================

"""
➡️ Implémenter la loss + gradient descente

🔧 À faire :
- Complète la loss
- Applique gradient descent sur W1 et W2
"""

def train(pairs, W1, W2, epochs=100, lr=0.01):
    for epoch in range(epochs):
        total_loss = 0
        for center, context in pairs:
            x = one_hot_encode(center, word_to_ix)
            y_true = one_hot_encode(context, word_to_ix)

            y_pred, h = forward(center, W1, W2)

            # Loss = cross entropy
            loss = -np.sum(y_true * np.log(y_pred + 1e-9))
            total_loss += loss

            # Backprop
            e = y_pred - y_true             # Erreur
            dW2 = np.outer(h, e)            # Grad sortie
            dW1 = np.outer(x, np.dot(W2, e)) # Grad entrée

            W1 -= lr * dW1
            W2 -= lr * dW2

        if epoch % 10 == 0:
            print(f"Epoch {epoch} → Loss: {total_loss:.4f}")

    return W1

W1_trained = train(pairs, W1, W2, epochs=100, lr=0.05)

In [ ]:
# ========================================
# 📈 9. Visualiser les embeddings
# ========================================

"""
➡️ Chaque mot est une ligne de W1
➡️ On réduit en 2D avec PCA pour visualiser

🔧 À faire :
- Lancer le plot
"""

def plot_embeddings(W1, ix_to_word):
    words = list(ix_to_word.values())
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(W1)

    plt.figure(figsize=(8,6))
    plt.scatter(reduced[:, 0], reduced[:, 1])
    for i, word in enumerate(words):
        plt.annotate(word, (reduced[i, 0], reduced[i, 1]))
    plt.title("🧭 Word Embeddings (PCA)")
    plt.grid(True)
    plt.show()

plot_embeddings(W1_trained, ix_to_word)

In [ ]:
# ========================================
# 🤔 10. Réflexion & Compréhension
# ========================================

"""
🧠 Questions à réfléchir :
- Est-ce que les mots similaires sont proches ?
- Est-ce que le contexte influence fortement les vecteurs ?
- Quelle est la différence si tu augmentes le corpus ?
- Comment ce modèle apprend à "regrouper" les idées ?

✏️ Notes perso :
- ...
"""

Ce que tu vas apprendre en le remplissant :
Étape	Ce que tu comprends
One-hot	Structure du vocabulaire
W1 / W2	Matrices d’apprentissage
Softmax	Interprétation des scores
Cross Entropy	Apprentissage supervisé
Embeddings	Représentation des mots
PCA	Visualisation